![](https://github.com/destination-earth/DestinE-DataLake-Lab/blob/main/img/DestinE-banner.jpg?raw=true)



# DEDL - HDA Tutorial - Queryables

<br> Author: EUMETSAT </br>

<div class="alert alert-block alert-success">
<h3>How to use the queryables API</h3>
<ol>
    <li> The queryables API. The API returns a list of variable terms that can be used for filtering the specified collection. </li>
    <li> How to filter a specific collection using the list of variable terms returned by the queryables API. </li> 
</ol>
</div>

This notebook demonstrates how to use the HDA (Harmonized Data Access) queryables API by sending a few HTTP requests to the API, using Python code.

The detailed HDA API and definition of each endpoint and parameters is available in the HDA Swagger UI at:

https://hda.data.destination-earth.eu/docs/

<div class="alert alert-block alert-warning">
<b> Prequisites: </b>
<li> For queryables API: none </li>
<li> For filtering collections : <a href="https://platform.destine.eu/"> DestinE user account</a> </li>
</div>

## Define some constants for the API URLs
In this section, we define the relevant constants, holding the URL strings for the different endpoints.

In this example, we search data for the dataset [CAMS European Air Quality Forecast](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview). 
The corresponding collection ID in HDA for this dataset is: *EO.ECMWF.DAT.CAMS_EUROPE_AIR_QUALITY_FORECASTS*.

In [187]:
# Use the Collection https://hda.data.destination-earth.eu/ui/dataset/EO.ESA.DAT.SENTINEL-2.MSI.L1C
COLLECTION_ID = "EO.ECMWF.DAT.CAMS_EUROPE_AIR_QUALITY_FORECASTS"

# Core API
HDA_API_URL = "https://hda.data.destination-earth.eu"

# STAC API
## Core
STAC_API_URL = f"{HDA_API_URL}/stac"

## Item Search
SEARCH_URL = f"{STAC_API_URL}/search"

## Collections
COLLECTIONS_URL = f"{STAC_API_URL}/collections"

## Queryables
QUERYABLES_URL = f"{STAC_API_URL}/queryables"
QUERYABLES_BY_COLLECTION_ID = f"{COLLECTIONS_URL}/{COLLECTION_ID}/queryables"
HDA_FILTERS =''

## HTTP Success
HTTP_SUCCESS_CODE = 200

## Import the relevant modules and define some useful functions
We start off by importing the relevant modules for DestnE authentication, HTTP requests, json handling and widgets.

In [188]:
import destinelab as deauth

In [189]:
from typing import Union
import requests
import json
import urllib.parse
from requests.auth import HTTPBasicAuth

import ipywidgets as widgets
from IPython.display import display, clear_output
import datetime

from rich.console import Console
import rich.table
import json

from IPython.display import JSON


Below useful functions for pretty printing and for demonstrating the queryables API.

In [190]:
def display_as_json(response: requests.Response) -> None:
    """Displays a HTTP request response as an interactive JSON in Jupyter Hub.
    
    Args:
        response (requests.Response): HTTP request response
    Returns:
        None
    """
    if not isinstance(response, requests.Response):
        raise TypeError(f"display_as_json expects a requests.Response parameter, got {type(response)}.")
    return JSON(json.loads(response.text))

def create_q_table(filters,params=None):
    table = rich.table.Table(title="Applicable filters", expand=True, show_lines=True)
    table.add_column("Description", style="cyan", justify="right")
    table.add_column("Type", style="violet", justify="right", no_wrap=True)
    table.add_column("enum", style="violet", justify="right")
    table.add_column("value", style="violet", justify="right", no_wrap=True)
    for filtername in filters.keys():
        if (params!=None and filtername not in params.keys()):
            continue
        enum=''
        if 'enum' in filters[filtername]:
           enum=' , ' .join(map(str,filters[filtername]["enum"]))
        value=''
        if'value' in filters[filtername]:
           value=json.dumps(filters[filtername]["value"])
        if'type' in filters[filtername]:
           typeq=json.dumps(filters[filtername]["type"])
        else:
            typeq=''
    
        table.add_row(filters[filtername]["description"],  typeq , enum, value)
    return table

# Function to fetch queryable properties for a given collection with optional params
def fetch_queryables(collection_name, params=None):
    url = f"{COLLECTIONS_URL}/{collection_name}/queryables"
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json().get('properties', {})
    else:
        return None

# Function to make an API call with the selected variable values
def fetch_variable_details(collection_name, params):
    url = f"{COLLECTIONS_URL}/{collection_name}/queryables"
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return None

    
def update_dropdowns(collection_name, params=None):
    properties = fetch_queryables(collection_name, params)
    global HDA_FILTERS
    
    with output_area:
        clear_output()
        if properties:
            print("Properties fetched successfully.")
            #print(json.dumps(properties, indent=2))
            table=create_q_table(properties, params)
            console = Console()
            console.print(table)
            if (params!=None):
                print("The parameters chosen can be translated in the following filters for the HDA query:\n" )
                HDA_FILTERS = {
                    key: {"eq": value}
                    for key, value in params.items()
                }
        else:
            print("Failed to fetch properties.")
            return
        
    # Preserve existing selected values
    selected_values = {prop: dropdown.value for prop, dropdown in dropdowns.items()}
    
    # Clear existing dropdowns
    dropdown_container.children = []
    dropdowns.clear()
    
    # Create new dropdowns for properties with enum values
    new_dropdowns = []
    for prop, details in properties.items():
        if details.get('type') == 'string' and 'enum' in details:
            options = details['enum']
            #if prop == 'levtype':
            options = [''] + options  # Add empty option for 'param' property

            dropdown = widgets.Dropdown(
                description=prop,
                options=options,
                value=selected_values.get(prop, options[0])  # Set previously selected value or default to the first option
            )
            dropdown.observe(on_value_change, names='value')
            dropdowns[prop] = dropdown
            new_dropdowns.append(dropdown)
                
    if new_dropdowns:
        dropdown_container.children = new_dropdowns
    else:
        with output_area:
            print("No properties with enum values found.")

def on_fetch_button_clicked(b):
    collection_name = collection_input.value    
    update_dropdowns(collection_name)

def on_value_change(change):
    collection_name = collection_input.value
    params = {prop: dropdown.value for prop, dropdown in dropdowns.items() if dropdown.value is not None}
    
    if params:
        details = fetch_variable_details(collection_name, params)
    
        with output_area:
            clear_output()
            print(json.dumps(details, indent=2))
    
    # Update dropdowns based on the new selection
    update_dropdowns(collection_name, params) 
    



## 1 - The queryables API

The applicable search filters for the selected collection can be found starting from the following link:
[CAMS European Air Quality Forecast - filter Options HDA](https://hda.data.destination-earth.eu/stac/collections/EO.ECMWF.DAT.CAMS_EUROPE_AIR_QUALITY_FORECASTS/queryables?provider=copernicus_atmosphere_data_store)

Run the cell below to see the pretty print of this first request.

In [191]:
display_as_json(requests.get(QUERYABLES_BY_COLLECTION_ID))

<IPython.core.display.JSON object>

The applicable filters are described under the section named 'properties'.

This section contains 
* the name of the filter, **description**,
* the filter **type**, 
* the possible filter values, **enum** (conditioned by the values selected for the other filters)
* and the the default (or chosen) **value** applied

We can print the'properties' section in a table to see the filters and the values applied by default when we perform a search for the given dataset.

In [192]:
filters_resp=requests.get(QUERYABLES_BY_COLLECTION_ID)
filters = filters_resp.json()["properties"]
table=create_q_table(filters)
console = Console()
console.print(table)

                                                Applicable filters                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃                                Description ┃     Type ┃                                       enum ┃      value ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│                                         ID │          │                                            │            │
├────────────────────────────────────────────┼──────────┼────────────────────────────────────────────┼────────────┤
│                                   Geometry │          │                                            │            │
├────────────────────────────────────────────┼──────────┼────────────────────────────────────────────┼────────────┤
│     Datetime - use parameters year, month, │          │                                            │            │
│             day, time instead if available │          │                                            │            │
├────────────────────────────────────────────┼──────────┼────────────────────────────────────────────┼────────────┤
│                                       BBox │  "array" │                                            │            │
├────────────────────────────────────────────┼──────────┼────────────────────────────────────────────┼────────────┤
│                                      model │ "string" │ chimere , dehm , emep , ensemble , euradim │ "ensemble" │
│                                            │          │ , gemaq , lotos , match , minni , mocage , │            │
│                                            │          │                            monarch , silam │            │
├────────────────────────────────────────────┼──────────┼────────────────────────────────────────────┼────────────┤
│                                     format │ "string" │                                            │     "grib" │
├────────────────────────────────────────────┼──────────┼────────────────────────────────────────────┼────────────┤
│                                   variable │ "string" │    alder_pollen , ammonia , birch_pollen , │  "ammonia" │
│                                            │          │    carbon_monoxide , dust , formaldehyde , │            │
│                                            │          │  glyoxal , grass_pollen , mugwort_pollen , │            │
│                                            │          │     nitrogen_dioxide , nitrogen_monoxide , │            │
│                                            │          │  non_methane_vocs , olive_pollen , ozone , │            │
│                                            │          │                  particulate_matter_10um , │            │
│                                            │          │                 particulate_matter_2.5um , │            │
│                                            │          │  peroxyacyl_nitrates , pm10_sea_salt_dry , │            │
│                                            │          │                           pm10_wildfires , │            │
│                                            │          │   pm2.5_anthropogenic_fossil_fuel_carbon , │            │
│                                            │          │  pm2.5_anthropogenic_wood_burning_carbon , │            │
│                                            │          │               pm2.5_total_organic_matter , │            │
│                                            │          │                           ragweed_pollen , │            │
│                                            │          │            residential_elementary_carbon , │            │
│                                            │          │              secondary_inorganic_aerosol , │            │
│                                            │          

The most useful way to exploit the queryables API is to have an helper for building a correct search request for the given dataset.

Calling the queryables API with the values chosen for filtering the selected dataset, the API replies with the applicable filters, conditioned by the chosen values. This means that, if you select a certain variable then the choice is narrowed down for other variables.
In this way the queryables API provides a valid tool for constructing the right query.

Below an interactive example to see that once you select a value for a property the choice is narrowed down for other variables. 

If you, for example, select type=forecast (https://hda.data.destination-earth.eu/stac/collections/EO.ECMWF.DAT.CAMS_EUROPE_AIR_QUALITY_FORECASTS/queryables?provider=copernicus_atmosphere_data_store&type=forecast) you will have only '00:00' in the enum for time instead of all the available hours. 

In [193]:
# Widgets
collection_input = widgets.Text(
    description='Collection:',
    value=COLLECTION_ID,
)

fetch_button = widgets.Button(description="Fetch Properties")
dropdown_container = widgets.VBox()
output_area = widgets.Output()

dropdowns = {}

# Event listeners
fetch_button.on_click(on_fetch_button_clicked)

# Layout
display(collection_input, fetch_button, dropdown_container, output_area)

Text(value='EO.ECMWF.DAT.CAMS_EUROPE_AIR_QUALITY_FORECASTS', description='Collection:')

Button(description='Fetch Properties', style=ButtonStyle())

VBox()

Output()

## 2 - Filtering a collection with the list returned by the queryable API

This section wil explain how to use the list of variable terms returned by the queryables API for filtering a specific dataset. 

### Build the query from the selected values
The parameters chosen in the previous steps can be translated in the following filters to add to the HDA queries.

In [194]:
print(json.dumps(HDA_FILTERS, indent=4))

{
    "model": {
        "eq": "chimere"
    },
    "variable": {
        "eq": "peroxyacyl_nitrates"
    },
    "type": {
        "eq": "forecast"
    },
    "time": {
        "eq": "00:00"
    },
    "level": {
        "eq": "5000"
    },
    "leadtime_hour": {
        "eq": "23"
    }
}


build the complete query

In [202]:
# The JSON objects containing the generic query parameters:
json1 = '{"collections": ["EO.ECMWF.DAT.CAMS_EUROPE_AIR_QUALITY_FORECASTS"], "datetime": "2024-04-01T00:00:00Z/2024-04-19T00:00:00Z"}'

# Convert JSON strings to Python dictionaries
dict1 = json.loads(json1)

# Include the filters selected in the previous steps inside the JSON containing the generic query parameters:
dict1['query'] = HDA_FILTERS

# Convert the merged dictionary back to a JSON string
query_json = json.dumps(dict1, indent=4)

print(query_json)

{
    "collections": [
        "EO.ECMWF.DAT.CAMS_EUROPE_AIR_QUALITY_FORECASTS"
    ],
    "datetime": "2024-04-01T00:00:00Z/2024-04-19T00:00:00Z",
    "query": {
        "model": {
            "eq": "chimere"
        },
        "variable": {
            "eq": "peroxyacyl_nitrates"
        },
        "type": {
            "eq": "forecast"
        },
        "time": {
            "eq": "00:00"
        },
        "level": {
            "eq": "5000"
        },
        "leadtime_hour": {
            "eq": "23"
        }
    }
}


### Obtain Authentication Token
To perform a query on HDA we need to be authenticated.

In [197]:
import json
import os
from getpass import getpass
import destinelab as deauth

DESP_USERNAME = input("Please input your DESP username or email: ")
DESP_PASSWORD = getpass("Please input your DESP password: ")

auth = deauth.AuthHandler(DESP_USERNAME, DESP_PASSWORD)
access_token = auth.get_token()
if access_token is not None:
    print("DEDL/DESP Access Token Obtained Successfully")
else:
    print("Failed to Obtain DEDL/DESP Access Token")

auth_headers = {"Authorization": f"Bearer {access_token}"}

Please input your DESP username or email:  eum-dedl-user
Please input your DESP password:  ········


Response code: 200
DEDL/DESP Access Token Obtained Successfully


## Search

In [203]:
response = requests.post("https://hda.data.destination-earth.eu/stac/search", headers=auth_headers, json= json.loads(query_json) )
#print(response)
# Requests to ADS data always return a single item containing all the requested data
product = response.json()["features"][0]
product

{'stac_version': '1.0.0',
 'stac_extensions': ['https://stac-extensions.github.io/sar/v1.0.0/schema.json',
  'https://stac-extensions.github.io/order/v1.1.0/schema.json'],
 'id': 'CAMS_EU_AIR_QUALITY_FORECAST_20240401_20240418_708ed0b28e8f6f01326a712d5baf01145e8daa28',
 'bbox': [-180.0, -90.0, 180.0, 90.0],
 'geometry': {'type': 'Polygon',
  'coordinates': [[[180.0, -90.0],
    [180.0, 90.0],
    [-180.0, 90.0],
    [-180.0, -90.0],
    [180.0, -90.0]]]},
 'type': 'Feature',
 'collection': 'EO.ECMWF.DAT.CAMS_EUROPE_AIR_QUALITY_FORECASTS',
 'properties': {'providers': [{'name': 'copernicus_atmosphere_data_store',
    'description': 'Copernicus Atmosphere Data Store',
    'roles': ['host'],
    'url': 'https://ads.atmosphere.copernicus.eu/',
    'priority': 0}],
  'datetime': '2024-04-01',
  'start_datetime': '2024-04-01',
  'end_datetime': '2024-04-18',
  'license': 'proprietary',
  'instruments': [None],
  'sar:product_type': 'CAMS_EU_AIR_QUALITY_FORECAST',
  'order:status': 'orderable

Once we have found the product we can obtain the URL for downloading it:


In [204]:
# DownloadLink is an asset representing the whole product
download_url = product["assets"]["downloadLink"]["href"]
print(download_url )

http://hda.data.destination-earth.eu/stac/collections/EO.ECMWF.DAT.CAMS_EUROPE_AIR_QUALITY_FORECASTS/items/CAMS_EU_AIR_QUALITY_FORECAST_20240401_20240418_708ed0b28e8f6f01326a712d5baf01145e8daa28/download?provider=copernicus_atmosphere_data_store&_dc_qs=%257B%2522date%2522%253A%2B%25222024-04-01%252F2024-04-18%2522%252C%2B%2522format%2522%253A%2B%2522grib%2522%252C%2B%2522leadtime_hour%2522%253A%2B23%252C%2B%2522level%2522%253A%2B5000%252C%2B%2522model%2522%253A%2B%2522chimere%2522%252C%2B%2522time%2522%253A%2B%252200%253A00%2522%252C%2B%2522type%2522%253A%2B%2522forecast%2522%252C%2B%2522variable%2522%253A%2B%2522peroxyacyl_nitrates%2522%257D
